### Least Square Monte Carlo (LSM)
This python notebook implements the algrithm to price american options based on the very well known paper by Longstaff and Schwartz. The pricing and optimal exercise of American options is a challenging problems in derivatives finance, particularly when more than one factor affects the value of the option.

At any exercise time, the holder of an American option optimally compares the payoff from immediate exercise with the expected payoff from continuation, and then exercises if the immediate payoff is higher. Thus the optimal exercise strategy is fundamentally determined by the conditional expectation of the payoff from continuing to keep the option alive. The key insight underlying of LSM approach is that this conditional expectation can be estimated from the cross-sectional information in the simulation by using least squares. 

Specifically, 

* Regress the ex post realized payoffs from continuation on functions of the values of the state variables. 
* The fitted value from above regression provides a direct estimate of the conditional expectation function. 
* By estimating the conditional expectation function for each exercise date, we obtain a complete specification of the optimal exercise strategy along each path. 
* With above specification, American options can then be valued accurately by simulation. 

Longstaff and Schwartz refer to this technique as the least squares Monte Carlo (LSM) approach.

### Intution through a simple example
Below a simple example discussed in the paper to convey the intution of LSM approach. Consider an American put option on a
share of non-dividend-paying stock. The put option is exercisable at a strike price of 1.10 at times 1 , 2, and 3, where time three is the final expiration date of the option. The riskless rate is 6%. For simplicity, illustration shows the algorithm using only eight sample paths for the price of the stock. These sample paths are generated under the risk-neutral measure and are shown in the following matrix.

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

pd.set_option('display.colheader_justify', 'center')  # Center align column headers
pd.set_option('display.width', None)  # Adjust for wider output if needed

# Data
k = 1.1
r = 0.06
samplePaths = {
    'Path': ['1', '2', '3', '4', '5', '6', '7', '8'],
    't = 0': [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
    't = 1': [1.09, 1.16, 1.22, 0.93, 1.11, 0.76, 0.92, 0.88],
    't = 2': [1.08, 1.26, 1.07, 0.97, 1.56, 0.77, 0.84, 1.22],
    't = 3': [1.34, 1.54, 1.03, 0.92, 1.52, 0.90, 1.01, 1.34]
}

# Create DataFrame
samplePaths = pd.DataFrame(samplePaths)

# Display the DataFrame
print("Stock price paths \n")
print(samplePaths.to_string(index=False))


Stock price paths 

Path  t = 0  t = 1  t = 2  t = 3
 1    1.0    1.09   1.08   1.34 
 2    1.0    1.16   1.26   1.54 
 3    1.0    1.22   1.07   1.03 
 4    1.0    0.93   0.97   0.92 
 5    1.0    1.11   1.56   1.52 
 6    1.0    0.76   0.77   0.90 
 7    1.0    0.92   0.84   1.01 
 8    1.0    0.88   1.22   1.34 


The algorithm is recursive and starts first by getting the payoffs at the expiry in the above case at point t = 3. Cash flow metrics at time 3 is calculated below.

In [2]:
cashFlowMetricsTime3 = pd.DataFrame()
cashFlowMetricsTime3['Path'] =  samplePaths['Path']
cashFlowMetricsTime3['t = 1'] = '-'
cashFlowMetricsTime3['t = 2'] = '-'
cashFlowMetricsTime3['t = 3'] = (k - samplePaths['t = 3']).clip(0)

# Display the DataFrame
print("Cash flow matrix at time 3 \n")
print(cashFlowMetricsTime3.to_string(index=False))

Cash flow matrix at time 3 

Path t = 1 t = 2  t = 3
 1     -     -    0.00 
 2     -     -    0.00 
 3     -     -    0.07 
 4     -     -    0.18 
 5     -     -    0.00 
 6     -     -    0.20 
 7     -     -    0.09 
 8     -     -    0.00 


If the put is in the money at time 2, the optionholder must then decide whether to exercise the option immediately or continue the option's life until the final expiration date at time 3. From the stock-price matrix, there are only five paths for which the option is in the money at time 2. Let $X$ denote the stock prices at time 2 for these five paths and $Y$ denote the corresponding discounted cash flows received at time 3 if the put is not exercised at time 2. Only in-the-money paths are used since it allows to better estimate the conditional expectation function in the region where exercise is relevant and significantly improves the efficiency of the algorithm. The vectors $X$ and $Y$ are given by the nondashed entries below.

In [3]:
discountFactorTime2 = round(np.exp(-r*1), 5)
#get X and Y at time 2
regressionTime2 = pd.DataFrame()
regressionTime2['Path'] = samplePaths['Path']
regressionTime2['Y'] = round(cashFlowMetricsTime3['t = 3'],2).astype(str) + ' x ' +  str(discountFactorTime2)
regressionTime2['X'] = samplePaths['t = 2']

#check if option is in the money at time 2 and consider only those paths
regressionTime2[['Y', 'X']] = regressionTime2.apply(lambda row: row[['Y', 'X']] if row['X'] < k else ['-'] * 2, axis=1)



# Display the DataFrame
print("Regression at time 2 \n")
print(regressionTime2.to_string(index=False, header=True, justify='center'))

Regression at time 2 

Path       Y          X  
 1    0.0 x 0.94176  1.08
 2                -     -
 3   0.07 x 0.94176  1.07
 4   0.18 x 0.94176  0.97
 5                -     -
 6    0.2 x 0.94176  0.77
 7   0.09 x 0.94176  0.84
 8                -     -


To estimate the expected cash flow from continuing the option's life conditional on the stock price at time 2, regress $Y$ on a constant, $X$, and $X^2$.

In [4]:

# working with a copy of the DataFrame
regressionTime2Final = regressionTime2.copy()

# Prepare the data for regression
regressionTime2Final['Y'] = cashFlowMetricsTime3['t = 3'] * discountFactorTime2

# Drop samples where option is out of the money at time 2
regressionTime2Final = regressionTime2Final[regressionTime2Final['X'] != '-']

# Convert 'Y' and 'X' to numeric using .loc to avoid SettingWithCopyWarning
regressionTime2Final.loc[:, 'Y'] = pd.to_numeric(regressionTime2Final['Y'], errors='coerce')
regressionTime2Final.loc[:, 'X'] = pd.to_numeric(regressionTime2Final['X'], errors='coerce')

# Create 'X^2' (squared term)
regressionTime2Final.loc[:, 'X_Squared'] = regressionTime2Final['X'] ** 2

# Define the independent variables (including constant)
X = regressionTime2Final[['X', 'X_Squared']]
X = sm.add_constant(X)  # Adds the constant term to the model

# Define the dependent variable
y = regressionTime2Final['Y']

# Perform the regression
model = sm.OLS(y, X).fit()

# Print the summary of the regression
#print(model.summary())

#get the final result
interceptTime2 = model.params['const']
coefficient_X_Time2 = model.params['X']
coefficient_X_squared_Time2 = model.params['X_Squared']


In [5]:
# Format the coefficients with proper signs
def format_coefficient(coef, is_first=False):
    if coef < 0:
        return f"- {abs(coef):.3f}"  # Negative coefficients
    elif is_first:
        return f"{coef:.3f}"  # First coefficient, no leading "+"
    else:
        return f"+ {coef:.3f}"  # Positive coefficients after the first one

# Create a LaTeX formula using Python's f-string to insert the variable values
conditionaExpectation = f"$E[Y|X] = {format_coefficient(interceptTime2, is_first = True)} {format_coefficient(coefficient_X_Time2)} X {format_coefficient(coefficient_X_squared_Time2)} X^2$"

print("The condition expectation function using the result of the regression is : ")
# Display the formula dynamically in the notebook using Markdown
from IPython.display import display, Markdown
display(Markdown(conditionaExpectation))

The condition expectation function using the result of the regression is : 


$E[Y|X] = - 1.070 + 2.983 X - 1.814 X^2$

In [6]:

#check optimal exersice check using above conditional expectaions and immediate payoff
optimalExercise2 = pd.DataFrame()
optimalExercise2['Path'] = samplePaths['Path']
optimalExercise2['Exercise'] = round(k - samplePaths['t = 2'],2)
optimalExercise2['Continuation'] = round(interceptTime2 + coefficient_X_Time2 * samplePaths['t = 2'] + coefficient_X_squared_Time2 * (samplePaths['t = 2'] ** 2),4)
optimalExercise2['Exercise > Continuation'] = np.where(optimalExercise2['Exercise'] > optimalExercise2['Continuation'], "Yes" , "No")

#discrad where option is out of the money
optimalExercise2['Exercise'] = np.where(optimalExercise2['Exercise'] < 0 , "-", optimalExercise2['Exercise'])
optimalExercise2['Continuation'] = np.where(optimalExercise2['Exercise'] == "-" , "-", optimalExercise2['Continuation'])
optimalExercise2['Exercise > Continuation'] = np.where(optimalExercise2['Exercise'] == "-" , "-", optimalExercise2['Exercise > Continuation'])


# Display the DataFrame
print("Optimal early exercise decision at time 2 \n")
print(optimalExercise2.to_string(index=False, header=True, justify='center'))

Optimal early exercise decision at time 2 

Path Exercise Continuation Exercise > Continuation
 1     0.02      0.0367               No          
 2        -           -                -          
 3     0.03      0.0459               No          
 4     0.13      0.1175              Yes          
 5        -           -                -          
 6     0.33       0.152              Yes          
 7     0.26      0.1564              Yes          
 8        -           -                -          


It is optimal to exercise option at time 4, 5 and 7 at time 2. Now the we will modify the cash flow matrix both at time 2 and 3 based on the above results. Specifically if if it is optimal to exercse at time 2 then we will make the corresponding non zero entries at time 3 zero as the option is exercized at time 2 and there are no further cash flows.

In [7]:
cashFlowMetricsTime2 = pd.DataFrame()
cashFlowMetricsTime2['Path'] =  samplePaths['Path']
cashFlowMetricsTime2['t = 1'] = '-'
cashFlowMetricsTime2['t = 2'] = np.where(optimalExercise2['Exercise > Continuation'] == "Yes", (k - samplePaths['t = 2']).clip(0), 0)
cashFlowMetricsTime2['t = 3'] = np.where(optimalExercise2['Exercise > Continuation'] == "Yes", 0, cashFlowMetricsTime3['t = 3'])

# Display the DataFrame
print("Cash flow matrix at time 2 \n")
print(cashFlowMetricsTime2.to_string(index=False))

Cash flow matrix at time 2 

Path t = 1  t = 2  t = 3
 1     -    0.00   0.00 
 2     -    0.00   0.00 
 3     -    0.00   0.07 
 4     -    0.13   0.00 
 5     -    0.00   0.00 
 6     -    0.33   0.00 
 7     -    0.26   0.00 
 8     -    0.00   0.00 


Proceeding recursively, we next examine whether the option should be exercised at time 1. From the stock price matrix, there are again five paths where the option is in the money at time 1. For these paths, we again define Y as the discounted value of subsequent option cash flows. Note that in defining Y, we use actual realized cash flows along each path; we do not
use the conditional expected value of Y estimated at time 2 in defining Y at time 1. Discounting back the conditional expected
value rather than actual cash flows can lead to an upward bias in the value of the option.

Since the option can only be exercised once, future cash flows occur at either time 2 or time 3, but not both. Cash flows received at time 2 are discounted back one period to time 1, and any cash flows received at time 3 are discounted back two periods to time 1. Similarly X represents the stock prices at time 1 for the paths where the option is in the money. The vectors X and Y are given by the nondashed elements in the following matrix.

In [8]:
discountFactorTime2 = round(np.exp(-r*1), 5)
discountFactorTime3 = round(np.exp(-r*2), 5)
#get X and Y at time 2
regressionTime1 = pd.DataFrame()
regressionTime1['Path'] = samplePaths['Path']
regressionTime1['Y'] = np.where(cashFlowMetricsTime2['t = 3'] != 0, 
                                round(cashFlowMetricsTime2['t = 3'],2).astype(str) + ' x ' +  str(discountFactorTime3),
                                round(cashFlowMetricsTime2['t = 2'],2).astype(str) + ' x ' +  str(discountFactorTime2))
regressionTime1['X'] = samplePaths['t = 1']

#check if option is in the money at time 2 and consider only those paths
regressionTime1[['Y', 'X']] = regressionTime1.apply(lambda row: row[['Y', 'X']] if row['X'] < k else ['-'] * 2, axis=1)



# Display the DataFrame
print("Regression at time 1 \n")
print(regressionTime1.to_string(index=False, header=True, justify='center'))

Regression at time 1 

Path       Y          X  
 1    0.0 x 0.94176  1.09
 2                -     -
 3                -     -
 4   0.13 x 0.94176  0.93
 5                -     -
 6   0.33 x 0.94176  0.76
 7   0.26 x 0.94176  0.92
 8    0.0 x 0.94176  0.88


In [9]:
#Regression step
# working with a copy of the DataFrame
regressionTime1Final = regressionTime1.copy()

# Prepare the data for regression
regressionTime1Final['Y'] = np.where(cashFlowMetricsTime2['t = 3'] != 0, 
                                cashFlowMetricsTime2['t = 3'] * discountFactorTime3,
                                cashFlowMetricsTime2['t = 2'] * discountFactorTime2)

# Drop samples where option is out of the money at time 2
regressionTime1Final = regressionTime1Final[regressionTime1Final['X'] != '-']

# Convert 'Y' and 'X' to numeric using .loc to avoid SettingWithCopyWarning
regressionTime1Final.loc[:, 'Y'] = pd.to_numeric(regressionTime1Final['Y'], errors='coerce')
regressionTime1Final.loc[:, 'X'] = pd.to_numeric(regressionTime1Final['X'], errors='coerce')

# Create 'X^2' (squared term)
regressionTime1Final.loc[:, 'X_Squared'] = regressionTime1Final['X'] ** 2

# Define the independent variables (including constant)
X = regressionTime1Final[['X', 'X_Squared']]
X = sm.add_constant(X)  # Adds the constant term to the model

# Define the dependent variable
y = regressionTime1Final['Y']

# Perform the regression
model = sm.OLS(y, X).fit()

# Print the summary of the regression
#print(model.summary())

#get the final result
interceptTime1 = model.params['const']
coefficient_X_Time1 = model.params['X']
coefficient_X_squared_Time1 = model.params['X_Squared']

# Format the coefficients with proper signs
def format_coefficient(coef, is_first=False):
    if coef < 0:
        return f"- {abs(coef):.3f}"  # Negative coefficients
    elif is_first:
        return f"{coef:.3f}"  # First coefficient, no leading "+"
    else:
        return f"+ {coef:.3f}"  # Positive coefficients after the first one

# Create a LaTeX formula using Python's f-string to insert the variable values
conditionaExpectationTime1 = f"$E[Y|X] = {format_coefficient(interceptTime1, is_first = True)} {format_coefficient(coefficient_X_Time1)} X {format_coefficient(coefficient_X_squared_Time1)} X^2$"

print("The condition expectation function using the result of the regression is : ")
# Display the formula dynamically in the notebook using Markdown
from IPython.display import display, Markdown
display(Markdown(conditionaExpectationTime1))

The condition expectation function using the result of the regression is : 


$E[Y|X] = 2.038 - 3.335 X + 1.356 X^2$

In [10]:

#check optimal exersice check using above conditional expectaions and immediate payoff
optimalExercise1 = pd.DataFrame()
optimalExercise1['Path'] = samplePaths['Path']
optimalExercise1['Exercise'] = round(k - samplePaths['t = 1'],2)
optimalExercise1['Continuation'] = round(interceptTime1 + coefficient_X_Time1 * samplePaths['t = 1'] + coefficient_X_squared_Time1 * (samplePaths['t = 1'] ** 2),4)
optimalExercise1['Exercise > Continuation'] = np.where(optimalExercise1['Exercise'] > optimalExercise1['Continuation'], "Yes" , "No")

#discrad where option is out of the money
optimalExercise1['Exercise'] = np.where(optimalExercise1['Exercise'] < 0 , "-", optimalExercise1['Exercise'])
optimalExercise1['Continuation'] = np.where(optimalExercise1['Exercise'] == "-" , "-", optimalExercise1['Continuation'])
optimalExercise1['Exercise > Continuation'] = np.where(optimalExercise1['Exercise'] == "-" , "-", optimalExercise1['Exercise > Continuation'])


# Display the DataFrame
print("Optimal early exercise decision at time 1 \n")
print(optimalExercise1.to_string(index=False, header=True, justify='center'))

Optimal early exercise decision at time 1 

Path Exercise Continuation Exercise > Continuation
 1     0.01      0.0135               No          
 2        -           -                -          
 3        -           -                -          
 4     0.17      0.1087              Yes          
 5        -           -                -          
 6     0.34      0.2861              Yes          
 7     0.18       0.117              Yes          
 8     0.22      0.1528              Yes          


Based on the above results we can modify our cash flow matrix obtained at time 2. Make all the entries at later time points are made equal to 0 f early exercise is possible at time 1.

In [11]:
cashFlowMetricsTime1 = pd.DataFrame()
cashFlowMetricsTime1['Path'] =  samplePaths['Path']
cashFlowMetricsTime1['t = 1'] = np.where(optimalExercise1['Exercise > Continuation'] == "Yes", (k - samplePaths['t = 1']).clip(0), 0)
cashFlowMetricsTime1['t = 2'] = np.where(optimalExercise1['Exercise > Continuation'] == "Yes", 0,
                                np.where(optimalExercise2['Exercise > Continuation'] == "Yes", 
                                         (k - samplePaths['t = 2']).clip(0), 0))
cashFlowMetricsTime1['t = 3'] = np.where((optimalExercise1['Exercise > Continuation'] == "Yes") & (optimalExercise2['Exercise > Continuation'] == "Yes"),
                                         0, cashFlowMetricsTime3['t = 3'])

# Display the DataFrame
print("Option Cash flow matrix \n")
print(cashFlowMetricsTime1.to_string(index=False))

#conver this matrix to stoppin time rule
print("\n\n Stopping rule \n")
cashFlowMetricsTime1.set_index('Path', inplace=True)
stoppingRule = (cashFlowMetricsTime1 != 0).astype(int)
print(stoppingRule.to_string(index=True))

Option Cash flow matrix 

Path  t = 1  t = 2  t = 3
 1    0.00   0.0    0.00 
 2    0.00   0.0    0.00 
 3    0.00   0.0    0.07 
 4    0.17   0.0    0.00 
 5    0.00   0.0    0.00 
 6    0.34   0.0    0.00 
 7    0.18   0.0    0.00 
 8    0.22   0.0    0.00 


 Stopping rule 

      t = 1  t = 2  t = 3
Path                     
1       0      0      0  
2       0      0      0  
3       0      0      1  
4       1      0      0  
5       0      0      0  
6       1      0      0  
7       1      0      0  
8       1      0      0  


In [12]:
#after identifying final cash flow matrix using the stopping rule we an now get the price of American option
am_option_price = np.average(cashFlowMetricsTime1['t = 1'] * np.exp(-r*1) + cashFlowMetricsTime1['t = 2'] * np.exp(-r*2) + 
                  cashFlowMetricsTime1['t = 3'] * np.exp(-r*3) )
#get European option price based on cash flows at time 3 only wuthout early exercise
eu_option_price = np.average(cashFlowMetricsTime3['t = 3'] * np.exp(-r*3) )
print("Price of American option price is : " + str(round(am_option_price, 4)))
print("\nPrice of European option price is : " + str(round(eu_option_price, 4)))

Price of American option price is : 0.1144

Price of European option price is : 0.0564


We will try here to reproduce the Table 1 in the paper, mainly closed form european and simulated american put prices. Same setting are used, total 100,000 (50,000 plus 50,000 anthithetic), striek is 40, risk free rate is 0.06%.

In [20]:
%run AmOption
S = [36, 38, 40, 42, 44]
sigmas = [0.2, 0.4]
Ts = [1, 2]
r = 0.06
K = 40
paths = 50000
#create empty dataframe to hold data
results_table1 = pd.DataFrame(columns=['S', 'sigma', 'T', 'Closed Form European', 'Simulated American', 'Difference', '(s.e.)'])

# Print the header
print(f"{'S':<{5}} {'Sigma':<{7}} {'T':<{5}} {'Closed form European':<{25}} {'Simulated American':<{20}} {'Difference':<{10}} {'(s.e.)':<{5}}")

# Print a separator with correct number of dashes
print(f"{'-' * 5} {'-' * 7} {'-' * 5} {'-' * 25} {'-' * 20} {'-' * 10} {'-' * 5}")

# # Print the data
for s0 in S:
    for sigma in sigmas:
        for T in Ts:
            steps = T * 50
            #create stock with spot, vol and RFR
            stock1 = Stock(s0, sigma, r)
            #create option object
            Option1 = Option(T, steps, K)
            #create BS model object with required number of paths and steps for above stock
            Model = BSModel(stock1, Option1, paths, steps)
            #create option now
            EurOption1 = EurOption(stock1, Option1)
            #generate random variables
            randomNums = Model.generateRandonNumbers(0)
            #generate paths now
            simulationPaths = Model.generateSamplePaths(randomNums)
            simulationPathsAnt = Model.generateSamplePaths(-randomNums)

            #get analytic european put option price  
            eurPutOptPriceByAnalytic = round(EurOption1.priceByBSAnalytic("put"), 3)
            
            #create american option now
            AmOption1 = AmOption(stock1, Option1, Model)
            dt = AmOption1.getDt()
            #print(dt)
            V, payoffMatrix, exerciseDecisionMatrix, results_dict = AmOption1.priceAmByLSMC(simulationPaths, dt)
            V1, payoffMatrix1, exerciseDecisionMatrix1, results_dict1 = AmOption1.priceAmByLSMC(simulationPathsAnt, dt)
            amOptionPrice = round((V+V1)/2,3)
            
            #difference between european and americal call option
            difference = round(amOptionPrice - eurPutOptPriceByAnalytic,3)
            #calculate s.e.
            arr = np.append(np.sum(payoffMatrix, axis = 1),(np.sum(payoffMatrix1, axis = 1)))
            se = round(np.std(arr, ddof=1) / np.sqrt(len(arr)),3)
            
            #Add results to datfrme New row as DataFrame
            # New row to be added as DataFrame
            new_row = pd.DataFrame({
                'S': [s0], 
                'sigma': [sigma], 
                'T': [T], 
                'Closed Form European': [eurPutOptPriceByAnalytic], 
                'Simulated American': [amOptionPrice], 
                'Difference': [difference], 
                '(s.e.)': [se]
            })

            # Concatenate the new row to the existing DataFrame
            results_table1 = pd.concat([results_table1, new_row], ignore_index=True)

            
            
            print(f"{s0:<5} {sigma:<7} {T:<5} {eurPutOptPriceByAnalytic:<25} {amOptionPrice:<20} {difference:<10} {se:<5}")

S     Sigma   T     Closed form European      Simulated American   Difference (s.e.)
----- ------- ----- ------------------------- -------------------- ---------- -----
36    0.2     1     3.844                     4.47                 0.626      0.009
36    0.2     2     3.763                     4.812                1.049      0.011
36    0.4     1     6.711                     7.086                0.375      0.019
36    0.4     2     7.7                       8.494                0.794      0.023
38    0.2     1     2.852                     3.244                0.392      0.009
38    0.2     2     2.991                     3.728                0.737      0.011
38    0.4     1     5.834                     6.135                0.301      0.019
38    0.4     2     6.979                     7.653                0.674      0.022
40    0.2     1     2.066                     2.306                0.24       0.009
40    0.2     2     2.356                     2.867                0.511   

Compare resuts obtained above and the results in original paper in the Table 1. Due to difference in radom numbers some small differences are expected.

In [29]:
Table1_Orig = pd.read_csv('Table1_OrigPaper.csv')
Table1_Orig.columns = [col + '_Orig' for col in Table1_Orig.columns]
compare_side_by_side = pd.concat([results_table1, Table1_Orig[['Samerican_Orig','s.e._Orig','CFEuropean_Orig','EEValue_Orig','DiEEValue_Orig']]], axis=1)
compare_side_by_side['OrigVsThis%'] = round((compare_side_by_side['Simulated American'] - compare_side_by_side['Samerican_Orig'])/compare_side_by_side['Simulated American'] * 100, 3)
compare_side_by_side

,S,sigma,T,Closed Form European,Simulated American,Difference,(s.e.),Samerican_Orig,s.e._Orig,CFEuropean_Orig,EEValue_Orig,DiEEValue_Orig,OrigVsThis%
0,36,0.2,1,3.844,4.470,0.626,0.009,4.472,0.010,3.844,0.634,0.006,-0.045
1,36,0.2,2,3.763,4.812,1.049,0.011,4.821,0.012,3.763,1.077,0.019,-0.187
2,36,0.4,1,6.711,7.086,0.375,0.019,7.091,0.020,6.711,0.390,0.010,-0.071
3,36,0.4,2,7.700,8.494,0.794,0.023,8.488,0.024,7.700,0.808,0.020,0.071
4,38,0.2,1,2.852,3.244,0.392,0.009,3.244,0.009,2.852,0.398,0.006,0.000
5,38,0.2,2,2.991,3.728,0.737,0.011,3.735,0.011,2.991,0.754,0.010,-0.188
6,38,0.4,1,5.834,6.135,0.301,0.019,6.139,0.019,5.834,0.314,0.009,-0.065
7,38,0.4,2,6.979,7.653,0.674,0.022,7.669,0.022,6.979,0.691,0.001,-0.209
8,40,0.2,1,2.066,2.306,0.240,0.009,2.313,0.009,2.066,0.248,0.001,-0.304
9,40,0.2,2,2.356,2.867,0.511,0.011,2.879,0.010,2.356,0.529,0.006,-0.419
